In [4]:
import geopandas as gpd
from shapely.geometry import box
import numpy as np

# 1. Load AOI polygon (as GeoDataFrame, from shapefile)
aoi = gpd.read_file("Limites_Amazonia_Legal_2022_shp/Limites_Amazonia_Legal_2022.shp").to_crs("EPSG:32721")# 2. Define grid parameters (in meters, UTM)
cell_size = 250  # meters

# 3. Get bounds of AOI in UTM
minx, miny, maxx, maxy = aoi.total_bounds

# 4. Generate grid
x_coords = np.arange(minx, maxx, cell_size)
y_coords = np.arange(miny, maxy, cell_size)

cells = []
for x in x_coords:
    for y in y_coords:
        cell_geom = box(x, y, x+cell_size, y+cell_size)
        cell_center = ((x + x + cell_size)/2, (y + y + cell_size)/2)
        cells.append({
            'cell_id': f"{int(cell_center[0])}_{int(cell_center[1])}",
            'geometry': cell_geom,
            'center': cell_center,
            'score': 0
        })

grid_gdf = gpd.GeoDataFrame(cells, crs="EPSG:32721")

# 5. Mask to AOI
grid_gdf = grid_gdf[grid_gdf.intersects(aoi.unary_union)]


KeyboardInterrupt: 

In [ ]:
# Sample: attach mean elevation (assuming 'elevation_raster' loaded with rasterio)
import rasterio
from rasterio.mask import mask

with rasterio.open("srtm_dem.tif") as src:
    for idx, row in grid_gdf.iterrows():
        geom = [row['geometry']]
        try:
            out_image, out_transform = mask(src, geom, crop=True)
            mean_elev = out_image[out_image != src.nodata].mean()
        except Exception:
            mean_elev = np.nan
        grid_gdf.at[idx, 'mean_elevation'] = mean_elev

# Distance to water: Use shapely's distance method, using centroids and water polygons
water_gdf = gpd.read_file("rivers_lakes.shp").to_crs(grid_gdf.crs)
water_union = water_gdf.unary_union

grid_gdf['dist_to_water'] = grid_gdf.centroid.apply(lambda p: p.distance(water_union))

# Known site buffer flag
sites_gdf = gpd.read_file("archaeo_sites.shp").to_crs(grid_gdf.crs)
sites_buffer = sites_gdf.buffer(500)  # 500m buffer, adjust as needed
sites_union = sites_buffer.unary_union

grid_gdf['site_buffer'] = grid_gdf.geometry.apply(lambda g: g.intersects(sites_union))
